# ICESat-2 Data Import and Visualization Demo
This notebook walks through how to use OpenOceans to read in ICESat-2 photon data and access various parameters/metadata. It also includes useful kml export and track data plotting capabilities.

Last updated: Jan 6 2023

In [1]:
# adding the open oceans folder to the import path
import sys
sys.path.append('..')

# import the bounding box module from open oceans
import openoceans as oo

import pandas as pd
import numpy as np
import os

## Load in ICESat-2 data from an existing h5 file.

In [2]:
# filepath = "path_to_your_h5_file"

filepath = '../sample_data/gbr_reef_ATL03_20210817155409_08401208_005_01.h5'

# which of the six beam data to read in
gt = 'gt1r' # or gt2r or gt3r or gt1l or gt2l or gt3l

# read in ICESat-2 photon data using Profile class
p = oo.Profile.from_h5(filepath, gt, verbose=False)

# print basic data about the profile
print(p)


TRACK DETAILS
    D/M/Y: 17/08/2021
    Reference Ground Track: 840
    Cycle: 12
    Region: 8
    Orbit Direction: DESCENDING
    Release: 005

BEAM DETAILS
    GT1R
    STRONG BEAM
    ATLAS SPOT 5 
    
ICESat-2 Profile: ATL03_20210817*_08401208_005
Photons Returned: 118088



You can also generate an easily programmatically modified and formatted filename using this metadata.

In [3]:
print(p.get_formatted_filename())

2021-08-17_rgt0840_cyc12_reg08_rel005_beamGT1R


## Using Profiles

The actual photon data is stored within the Profile's 'data' property.

In [4]:
p.data.head()

,is_land,signal,classification,lon,lat,time,rgt,cycle,region,track,...,delta_time,pair,geometry,ref_azimuth,ref_elev,dist_ph_along,surf_type_land,surf_type_ocean,surf_type_inland_water,geoid_z
0,0,0,0,149.531334,-19.421182,2021-08-17 15:59:12.712058,840,12,8,1,...,1.144512e+08,1,POINT (149.53133 -19.42118),-1.830126,1.561746,2.222997e+07,0,1,0,58.056393
1,0,0,0,149.531332,-19.421194,2021-08-17 15:59:12.712258,840,12,8,1,...,1.144512e+08,1,POINT (149.53133 -19.42119),-1.830126,1.561746,2.222997e+07,0,1,0,58.056393
2,0,0,0,149.531332,-19.421201,2021-08-17 15:59:12.712358,840,12,8,1,...,1.144512e+08,1,POINT (149.53133 -19.42120),-1.830126,1.561746,2.222997e+07,0,1,0,58.056393
3,0,0,0,149.531331,-19.421207,2021-08-17 15:59:12.712458,840,12,8,1,...,1.144512e+08,1,POINT (149.53133 -19.42121),-1.830126,1.561746,2.222997e+07,0,1,0,58.056393
4,0,0,0,149.531328,-19.421233,2021-08-17 15:59:12.712858,840,12,8,1,...,1.144512e+08,1,POINT (149.53133 -19.42123),-1.830126,1.561746,2.222998e+07,0,1,0,58.056393


Track metadata can be accessed programmatically through the Profile's 'info' property, or printed.

In [5]:
print(p.info)

# info is a dictionary, get specific parameters using keys as follows

# print(p.info['date'])

{'date': datetime.date(2021, 8, 17), 'rgt': 840, 'gtxx': 'gt1r', 'cycle': 12, 'region': 8, 'orbit_dir': 'DESCENDING', 'release': '005', 'beam_strength': 'strong', 'atlas_spot': 5, 'track_pair': 1}


## Clipping Shorelines
It can frequently be useful to trim away land surface returns when conducting bathymetric analyses. Each photon is assigned an `is_land` flag of 0 or 1 (otherwise -1) by comparing against the Natural Earth 10 land polygons in the shorelines directory.

In [9]:
# how many of each flag?
# recall:
# -1 : unnclassified (couldnt access shoreline data)
# 0: water
# 1: land

print(p.data.is_land.value_counts())

# subset dataframe accordingly
# ocean_data_only = p.data.iloc[np.where( p.data.is_land <= 0 )[0], :]

0    118088
Name: is_land, dtype: int64


Since the photon data has been massaged such that it's stored as a Pandas DataFrame, we can use exporting functions accordingly to save the data for additional analysis.

In [12]:
# Use programmatic way to get formatted filename for this specific ICESat-2 profile
csv_filename = p.get_formatted_filename() + '.csv'
csv_filepath = os.path.join('demo_data', csv_filename)

# Export photon data using DataFrame methods

# p.data.to_csv(csv_filepath)
# print('Successfully saved ' + csv_filename + ' to demo data directory.')

Successfully saved 2021-08-17_rgt0840_cyc12_reg08_rel005_beamGT1R.csv to demo data directory.


## Exporting ICESat-2 Data to KML

You can also export the track profile as a kml to view in Google Earth. This function only outputs the lon/lat of the track (no elevation data). To reduce memory load, data are subsampled at a spacing which can be specified by the user. 

By default, OpenOceans uses a 1km spacing between data points.

In [ ]:
# specifies filename as shown in previous cell, placing in data subdirectory
filename = os.path.join('data', p.get_formatted_filename()+'.kml')

# exports data with a point every 1000 meters
p.to_kml(filename=filename, meters=1000, verbose=False)

# p.to_kml() # places the kml file in the current working directory

## Spatial Subsetting

It's straightforward to spatially subset this photon data for your predefined area of interest, either at import or prgrammatically. (Note that the full H5 file/data must still be read into memory before spatial subsetting is applied.)

See demo 2 for using GeoAOI's to define your area of interest.